# Notebook 19: Frontend Changes - File Inputs vs Text Inputs

## 🎯 What You'll Learn

In our **Todo app**, the frontend was simple - text inputs and JSON requests. But when you need to handle **file uploads**, your React components need to work completely differently. This notebook shows you exactly what changes in your frontend code when you go from text-based forms to file-based forms.

## 📊 Todo Frontend vs PDF Frontend: Key Differences

| Aspect | Todo App | PDF App | Why Different? |
|---------|----------|---------|----------------|
| **Input Type** | `<input type="text">` | `<input type="file">` + text inputs | Need file selection |
| **Data Format** | `JSON.stringify()` | `FormData` for files, JSON for updates | Files can't be JSON |
| **User Interaction** | Type and submit | Select file, upload, then manage | Two-step process |
| **Display Logic** | Editable text fields | Links + metadata fields | Files vs text |
| **Error Handling** | Validation messages | Upload progress + validation | Complex feedback |
| **State Management** | Simple text state | File objects + metadata state | Multiple data types |

---

**💡 Key Insight**: File uploads require fundamentally different UI patterns because users need to select files from their computer, not just type text.

## Part 1: What We Had - Todo App Input Handling

### Todo App Input Component (Simple Text)

```javascript
// Todo app - Simple text input
export default function TodoList() {
  const [todos, setTodos] = useState([]);
  const [inputValue, setInputValue] = useState(''); // ← Just text

  // Simple text input handler
  const handleInputChange = (event) => {
    setInputValue(event.target.value); // ← Just store the text
  };

  // Simple form submission
  const handleSubmit = async (event) => {
    event.preventDefault();
    if (!inputValue.trim()) return;

    // Simple JSON request
    const response = await fetch(`${process.env.NEXT_PUBLIC_API_URL}/todos`, {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ // ← Simple JSON serialization
        name: inputValue,
        completed: false
      })
    });

    if (response.ok) {
      const newTodo = await response.json();
      setTodos([...todos, newTodo]);
      setInputValue(''); // ← Clear text input
    }
  };

  return (
    <form onSubmit={handleSubmit}>
      <input
        type="text"           // ← Simple text input
        value={inputValue}    // ← Controlled by text state
        onChange={handleInputChange}
        placeholder="What needs to be done?"
      />
      <button type="submit">Add Todo</button>
    </form>
  );
}
```

**Why this worked**: Todo creation is just text input → JSON → server. Simple and straightforward.

## Part 2: What Changes - PDF App File Upload

### PDF App Input Component (File + Text)

```javascript
// PDF app - File upload + metadata management
export default function PdfList() {
  const [pdfs, setPdfs] = useState([]);
  const [selectedFile, setSelectedFile] = useState(null); // ← File object, not text!
  const [filter, setFilter] = useState();

  // File input handler (completely different)
  const handleFileChange = (event) => {
    const file = event.target.files[0]; // ← Get File object from browser
    setSelectedFile(file); // ← Store File object, not text
    console.log('File selected:', file.name, file.size, file.type);
  };

  // File upload handler (much more complex)
  const handleUpload = async (event) => {
    event.preventDefault();
    
    // Validation
    if (!selectedFile) {
      alert("Please select file to load.");
      return;
    }
    
    // File type validation
    if (!selectedFile.type === 'application/pdf') {
      alert("Only PDF files are allowed.");
      return;
    }

    // FormData instead of JSON!
    const formData = new FormData();
    formData.append("file", selectedFile); // ← Attach file to FormData

    const response = await fetch(`${process.env.NEXT_PUBLIC_API_URL}/pdfs/upload`, {
      method: "POST",
      // NO Content-Type header! Browser sets it automatically for FormData
      body: formData // ← Send FormData, not JSON
    });

    if (response.ok) {
      const newPdf = await response.json();
      setPdfs([...pdfs, newPdf]);
      setSelectedFile(null); // ← Clear file selection
    } else {
      alert("Error loading file.");
    }
  };

  return (
    <form onSubmit={handleUpload}>
      <input
        type="file"          // ← File input, not text!
        accept=".pdf"        // ← Restrict to PDF files
        onChange={handleFileChange}
      />
      <button type="submit">Load PDF</button>
    </form>
  );
}
```

### Key Changes Highlighted:
1. **State type**: `string` → `File object`
2. **Input type**: `text` → `file`
3. **Data format**: `JSON.stringify()` → `FormData`
4. **Validation**: Text length → File type, size, existence
5. **Headers**: `Content-Type: application/json` → No header (browser sets it)
6. **Error handling**: Simple validation → Upload errors + file validation

## Part 3: Display Logic Changes - Text vs File Links

### Todo App Item Display (Editable Text)

```javascript
// Todo app - Simple editable text
export default function TodoItem({ todo, onChange, onDelete }) {
  return (
    <div className={styles.todoItem}>
      <input
        type="checkbox"
        checked={todo.completed}
        onChange={(e) => onChange(e, todo.id)}
      />
      <input
        type="text"                    // ← Editable text field
        value={todo.name}              // ← Display the todo text
        onChange={(e) => onChange(e, todo.id)}
      />
      <button onClick={() => onDelete(todo.id)}>Delete</button>
    </div>
  );
}
```

### PDF App Item Display (Links + Metadata)

```javascript
// PDF app - File links + editable metadata
import Image from 'next/image';

export default function PDFComponent({ pdf, onChange, onDelete }) {
  return (
    <div className={styles.pdfRow}>
      <input
        type="checkbox"
        name="selected"
        checked={pdf.selected}
        onChange={(e) => onChange(e, pdf.id)}
      />
      <input
        type="text"                    // ← Editable filename
        name="name"
        value={pdf.name}               // ← Display filename (not file content!)
        onChange={(e) => onChange(e, pdf.id)}
      />
      {/* NEW: View file link */}
      <a
        href={pdf.file}                // ← S3 URL for file download
        target="_blank"                // ← Open in new tab
        rel="noopener noreferrer"      // ← Security best practice
        className={styles.viewPdfLink}
      >
        <Image src="/document-view.svg" width="22" height="22" alt="View PDF" />
      </a>
      <button
        className={styles.deleteBtn}
        onClick={() => onDelete(pdf.id)}
      >
        <Image src="/delete-outline.svg" width="24" height="24" alt="Delete" />
      </button>
    </div>
  );
}
```

### Key Display Differences:
1. **Content**: Direct text → File metadata + link
2. **Interaction**: Edit text → Edit filename + view file
3. **Visual elements**: Text only → Icons for actions
4. **External links**: None → Links to S3 files
5. **Security**: Simple → `rel="noopener noreferrer"`

## Part 4: Update Logic Changes - Simple vs Complex

### Todo App Updates (Simple)

```javascript
// Todo app - Simple field updates
function handleTodoChange(e, id) {
  const target = e.target;
  const value = target.type === 'checkbox' ? target.checked : target.value;
  const name = target.name;
  
  // Update local state
  const updatedTodos = todos.map(todo => 
    todo.id === id ? { ...todo, [name]: value } : todo
  );
  setTodos(updatedTodos);
  
  // Update server - simple JSON
  const todoToUpdate = updatedTodos.find(todo => todo.id === id);
  updateTodo(todoToUpdate); // ← Send whole object as JSON
}

async function updateTodo(todo) {
  await fetch(`${process.env.NEXT_PUBLIC_API_URL}/todos/${todo.id}`, {
    method: 'PUT',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify(todo) // ← Simple JSON
  });
}
```

### PDF App Updates (More Complex)

```javascript
// PDF app - Debounced updates for better performance
import { debounce } from 'lodash';

// Debounced update function (performance optimization)
const debouncedUpdatePdf = useCallback(debounce((pdf, fieldChanged) => {
  updatePdf(pdf, fieldChanged);
}, 500), []); // ← Wait 500ms after user stops typing

function handlePdfChange(e, id) {
  const target = e.target;
  const value = target.type === 'checkbox' ? target.checked : target.value;
  const name = target.name;
  
  // Update local state immediately (for responsive UI)
  const copy = [...pdfs];
  const idx = pdfs.findIndex((pdf) => pdf.id === id);
  const changedPdf = { ...pdfs[idx], [name]: value };
  copy[idx] = changedPdf;
  setPdfs(copy);
  
  // Update server with debouncing
  debouncedUpdatePdf(changedPdf, name);
}

// Server update - must send full object due to backend design
async function updatePdf(pdf, fieldChanged) {
  const body_data = JSON.stringify(pdf); // ← Full object, not just changed field
  const url = process.env.NEXT_PUBLIC_API_URL + `/pdfs/${pdf.id}`;
  
  await fetch(url, {
    method: 'PUT',
    body: body_data,
    headers: { 'Content-Type': 'application/json' }
  });
}
```

### Why PDF Updates Are More Complex:
1. **Debouncing**: File apps often have more data, so we optimize API calls
2. **Full object**: Backend expects complete PDF object (including S3 URL)
3. **Performance**: Users might edit long filenames - don't want to call API on every keystroke
4. **State management**: More complex state with file URLs and metadata

## Part 5: FormData vs JSON - Key Technical Difference

### Todo App Data Sending (JSON)

```javascript
// Todo app - JSON data
const todoData = {
  name: "Learn React",
  completed: false
};

// Convert to JSON string
const jsonString = JSON.stringify(todoData);
// Result: '{"name":"Learn React","completed":false}'

// Send with correct header
fetch('/api/todos', {
  method: 'POST',
  headers: { 
    'Content-Type': 'application/json' // ← Tell server to expect JSON
  },
  body: jsonString // ← Send JSON string
});
```

### PDF App Data Sending (FormData)

```javascript
// PDF app - FormData for files
const fileInput = document.querySelector('input[type="file"]');
const selectedFile = fileInput.files[0];

// Create FormData (multipart/form-data format)
const formData = new FormData();
formData.append("file", selectedFile); // ← Attach file

// Can also add other data
formData.append("description", "My PDF file");
formData.append("category", "work");

// Send WITHOUT Content-Type header!
fetch('/api/pdfs/upload', {
  method: 'POST',
  // DON'T set Content-Type! Browser sets it automatically with boundary
  body: formData // ← Send FormData object
});
```

### What Happens Behind the Scenes:

**JSON Request:**
```
POST /api/todos HTTP/1.1
Content-Type: application/json
Content-Length: 42

{"name":"Learn React","completed":false}
```

**FormData Request:**
```
POST /api/pdfs/upload HTTP/1.1
Content-Type: multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Length: 12847

------WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Disposition: form-data; name="file"; filename="document.pdf"
Content-Type: application/pdf

[Binary PDF data here...]
------WebKitFormBoundary7MA4YWxkTrZu0gW--
```

### Why FormData Is Required:
- **Binary data**: JSON can't handle binary file content
- **File metadata**: Filename, content type included automatically
- **Streaming**: Large files can be streamed, not loaded into memory
- **Standard**: How browsers have always sent files

## Part 6: Error Handling and User Feedback

### Todo App Error Handling (Simple)

```javascript
// Todo app - Simple validation and errors
const handleSubmit = async (event) => {
  event.preventDefault();
  
  // Simple validation
  if (!inputValue.trim()) {
    alert('Please enter a todo item');
    return;
  }
  
  try {
    const response = await fetch('/api/todos', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ name: inputValue, completed: false })
    });
    
    if (response.ok) {
      // Success - clear input
      setInputValue('');
    } else {
      // Simple error
      alert('Failed to create todo');
    }
  } catch (error) {
    alert('Network error');
  }
};
```

### PDF App Error Handling (Complex)

```javascript
// PDF app - Complex validation and user feedback
const handleUpload = async (event) => {
  event.preventDefault();
  
  // File existence validation
  if (!selectedFile) {
    alert("Please select file to load.");
    return;
  }
  
  // File type validation
  if (selectedFile.type !== 'application/pdf') {
    alert("Only PDF files are allowed.");
    return;
  }
  
  // File size validation (10MB limit)
  const maxSize = 10 * 1024 * 1024; // 10MB
  if (selectedFile.size > maxSize) {
    alert("File is too large. Maximum size is 10MB.");
    return;
  }
  
  // Show upload progress (optional)
  setUploading(true);
  
  try {
    const formData = new FormData();
    formData.append("file", selectedFile);
    
    const response = await fetch(`${process.env.NEXT_PUBLIC_API_URL}/pdfs/upload`, {
      method: "POST",
      body: formData
    });
    
    if (response.ok) {
      const newPdf = await response.json();
      setPdfs([...pdfs, newPdf]);
      setSelectedFile(null);
      alert('File uploaded successfully!');
    } else {
      // Parse server error
      const errorData = await response.json();
      alert(`Upload failed: ${errorData.detail || 'Unknown error'}`);
    }
  } catch (error) {
    console.error('Upload error:', error);
    alert('Network error during upload');
  } finally {
    setUploading(false);
  }
};
```

### Error Types You Need to Handle:
1. **File selection**: No file selected
2. **File type**: Wrong file type (not PDF)
3. **File size**: File too large
4. **Server errors**: AWS errors, database errors
5. **Network errors**: Connection issues, timeouts
6. **Progress feedback**: Upload in progress, success/failure

## Part 7: State Management Complexity

### Todo App State (Simple)

```javascript
// Todo app - Simple state
function TodoList() {
  const [todos, setTodos] = useState([]);     // ← Array of todo objects
  const [inputValue, setInputValue] = useState(''); // ← Simple string
  const [filter, setFilter] = useState('all'); // ← Simple string
  
  // State is straightforward
  // todos: [{id: 1, name: "Learn React", completed: false}]
  // inputValue: "New todo text"
  // filter: "all" | "active" | "completed"
}
```

### PDF App State (More Complex)

```javascript
// PDF app - Complex state management
function PdfList() {
  const [pdfs, setPdfs] = useState([]);           // ← Array of PDF objects
  const [selectedFile, setSelectedFile] = useState(null); // ← File object or null
  const [filter, setFilter] = useState();         // ← undefined | true | false
  const [uploading, setUploading] = useState(false); // ← Upload progress state
  const didFetchRef = useRef(false);              // ← Ref to prevent double-fetching
  
  // State is more complex
  // pdfs: [{id: 1, name: "doc.pdf", selected: true, file: "https://s3.../uuid-doc.pdf"}]
  // selectedFile: File object {name: "doc.pdf", size: 123456, type: "application/pdf", ...}
  // filter: undefined (all) | true (selected) | false (not selected)
  // uploading: boolean for UI feedback
  // didFetchRef: prevents useEffect from running twice in development
}
```

### File Object Properties:
```javascript
// When user selects a file, you get a File object with:
const fileObject = {
  name: "document.pdf",           // Original filename
  size: 1048576,                  // File size in bytes
  type: "application/pdf",        // MIME type
  lastModified: 1640995200000,    // Timestamp
  // Plus methods:
  // .arrayBuffer() - get file as ArrayBuffer
  // .text() - get file as text (for text files)
  // .stream() - get file as ReadableStream
};
```

## Part 8: CSS and Styling Differences

### Todo App Styles (Simple)

```css
/* Todo app styles - focused on text */
.todoInput {
  width: 100%;
  padding: 12px;
  font-size: 16px;
  border: 1px solid #ddd;
}

.todoItem {
  display: flex;
  align-items: center;
  padding: 8px 0;
}

.todoText {
  flex-grow: 1;
  padding: 8px;
  border: none;
  background: transparent;
}
```

### PDF App Styles (More Complex)

```css
/* PDF app styles - files + actions */
.mainInputContainer {
  width: 100%;
  margin: 0px 0px 50px 0px;
}

.mainInput {
  padding: 5px;
  border: 1px solid #ccc;
  margin: auto;
  display: block;
  width: 560px;
  height: 20px;
  margin-bottom: 20px;
}

.loadBtn {
  background: none;
  border: 1px solid #ccc;
  padding: 5px 10px;
  border-radius: 4px;
  margin: auto;
  display: block;
}

.pdfRow {
  display: flex;
  flex-direction: row;
  align-items: center;
  margin: 10px 10px 0px 10px;
  padding: 5px;
}

.pdfInput {
  padding: 8px;
  border: 1px solid #ccc;
  width: calc(100% - 10px);
  height: 30px;
  margin: 0 10px 0px 10px;
  border-radius: 4px;
}

.viewPdfLink {
  margin-left: 10px;
  display: inline-flex;
  align-items: center;
}

.deleteBtn {
  background: none;
  border: 0;
  cursor: pointer;
  transition: color 0.3s ease;
}

.deleteBtn:hover {
  color: red;
}
```

### Styling Complexity Differences:
1. **File inputs**: Need custom styling (browsers style them differently)
2. **Action buttons**: View/download icons, delete buttons
3. **Layout**: More complex with multiple action items
4. **Icons**: Need icon assets for file actions
5. **States**: Upload progress, loading states

## 🎯 Key Takeaways

### What Changes in Your Frontend When Adding File Upload:

1. **Input Types**: Text inputs → File inputs + text inputs
2. **Data Format**: JSON → FormData for uploads, JSON for updates
3. **State Management**: Simple strings → File objects + metadata
4. **Validation**: Text validation → File type, size, existence validation
5. **User Feedback**: Simple alerts → Upload progress, detailed errors
6. **Display Logic**: Editable text → File links + editable metadata
7. **Performance**: Direct updates → Debounced updates
8. **Error Handling**: Simple → Multiple error types and sources

### Patterns You Can Reuse:

✅ **FormData pattern**: Use FormData for any file uploads  
✅ **File validation**: Always validate type, size, existence  
✅ **Debounced updates**: Prevent excessive API calls during editing  
✅ **File object handling**: Understand File API properties and methods  
✅ **External links**: Use `target="_blank"` and `rel="noopener noreferrer"`  
✅ **Upload feedback**: Provide clear success/error messages  

### Why These Changes Are Necessary:

- **Browser API**: File inputs work differently than text inputs
- **Data format**: Files can't be serialized to JSON
- **User experience**: File operations need different feedback
- **Performance**: File operations are slower, need optimization
- **Security**: File uploads have security implications
- **Functionality**: Files need viewing, not just editing

### Next Steps:

In **Notebook 20**, we'll explore the production complexity that file management introduces - from performance optimization to security considerations that don't exist in simple text-based applications.

---

**Remember**: The core React patterns from the Todo app still apply - we're just adapting them for file handling. Your knowledge of components, state, and events is still the foundation!